In [111]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.optim import Adam
import torch.optim as optim

import lightning as L
from lightning.pytorch import seed_everything
from torch.utils.data import DataLoader, Dataset, random_split

import os
# import cv2
# import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from lookup_dict import lookup_dict,id_to_name

In [2]:
root_path=""
dataset_path = os.path.join(root_path,"dataset")
images_dataset_path = os.path.join(dataset_path,"imageNet_images")
eeg_dataset_path = os.path.join(dataset_path,"eeg")

### TODO

- dataset load
- dataset split
- define model
- add classifier
- testing & accuracy


# Dataset

#### Merge all image folders to create list of images

In [3]:
import random

dir_list = list(os.walk(images_dataset_path))
# skip any unnecessary folders
start_idx = len(dir_list)-40
images_list = []
for sub_dir in dir_list[start_idx:]:
    # images_list+=sub_dir[2]
    images_list.extend(sub_dir[2])
    
images_list = [image_name.replace(".JPEG","") for image_name in images_list]

#### Split images list to 8:1:1

In [4]:
# random.shuffle(images_list)

images_total_size = len(images_list)
train_size = int(images_total_size*0.8)
val_size = int(images_total_size*0.1)
test_size = images_total_size-train_size-val_size

train_images = images_list[:train_size]
val_images = images_list[train_size:train_size+val_size]
test_images = images_list[-1*test_size:]

In [57]:
print(train_size)
print(val_size)
print(test_size)

1596
199
201


#### Make train, val, test dataset

In [282]:
eeg_dataset_name = "eeg_5_95_std.pth"
eeg_dataset = torch.load(os.path.join(eeg_dataset_path,eeg_dataset_name))

In [224]:
class CustomDataset(Dataset):
    def __init__(self,images_list) -> None:
        super().__init__()
        self.x_data = []
        self.y_data = []
        # self.y_data = [image_name.split("_")[0] for image_name in images_list]
        
        for eeg_segment in eeg_dataset['dataset']:
            for image_name in images_list:
                if eeg_dataset['images'][eeg_segment['image']] == image_name:
                    self.x_data.append(eeg_segment['eeg'][:,20:460])
                    # all_channel_list = np.array(eeg_segment['eeg'])
                    # self.x_data.append(torch.from_numpy(all_channel_list[:,40:480]))
                    # self.x_data.append(torch.FloatTensor([eeg_sequence[40:480] for eeg_sequence in eeg_segment['eeg']]))
                    class_id = image_name.split("_")[0]
                    self.y_data.append(lookup_dict[class_id])
                    # self.y_data.append(eeg_dataset['labels'][eeg_segment['label']])
                    break
                                    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return len(self.x_data)

In [290]:
d = eeg_dataset['dataset'][0]['eeg']
print(d.shape)
print(d.t().shape)
print(eeg_dataset['dataset'][0]['label'])

torch.Size([128, 500])
torch.Size([500, 128])
10


In [291]:
class EEGDataset(Dataset):
    def __init__(self,eeg_dataset_file_name="eeg_5_95_std.pth") -> None:
        super().__init__()
        loaded = torch.load(os.path.join(eeg_dataset_path,eeg_dataset_name))
        self.data = loaded['dataset']
        self.labels = loaded['labels']
        self.images = loaded['images']
        self.size = len(self.data)
        
    def __getitem__(self, idx):
        # t() -> transpose
        eeg = self.data[idx]['eeg'].t()
        eeg = eeg[20:460,:]
        
        label = self.data[idx]['label']
        return eeg, label
    
    def __len__(self):
        return self.size

In [ ]:
class Splitter(Dataset):
    def __init__(self,dataset,split_name='train') -> None:
        super().__init__()
        self.dataset = dataset
        
        loaded = torch.load(os.path.join(eeg_dataset_path,"block_splits_by_image_all.pth"))
        self.target_data_indices = loaded['splits'][0][split_name]
        # filter data that is too short
        self.target_data_indices = [i for i in self.target_data_idx if 450<=self.dataset.data[i]['eeg'].size(1)<=600]
        
        self.size = len(self.target_data_indices)
        
    def __getitem__(self, idx):
        eeg, label = self.dataset[self.target_data_indices[idx]]
        return eeg, label
    
    def __len__(self):
        return self.size

In [281]:
loaded = torch.load(os.path.join(eeg_dataset_path,"block_splits_by_image_all.pth"))
print(loaded['splits'])
print(len(loaded['splits']))
print(loaded['splits'][0].keys())
a = len(loaded['splits'][0]['train'])
b = len(loaded['splits'][0]['val'])
c = len(loaded['splits'][0]['test'])
print(a)
print(b)
print(c)
print(a+b+c)
print(loaded['splits'][2]['val'])
print(loaded['splits'][3]['val'])
print(loaded['splits'][4]['val'])
print(loaded['splits'][5]['val'])

data = eeg_dataset
print(val_dataset[loaded['splits'][0]['val'][0]])

sum=0
for subject in loaded['splits']:
    sum+=len(subject['train'])
print(sum)

#WHY NUMBER DIFFERENT FUCK

[{'train': [0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55, 57, 58, 60, 61, 62, 63, 66, 68, 69, 70, 71, 72, 73, 74, 77, 78, 79, 83, 84, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 110, 112, 113, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 135, 137, 138, 140, 141, 142, 143, 145, 146, 147, 149, 150, 151, 152, 153, 156, 157, 159, 161, 163, 164, 166, 168, 169, 173, 174, 175, 176, 177, 179, 180, 182, 185, 186, 187, 188, 189, 190, 191, 194, 195, 199, 200, 202, 208, 209, 211, 212, 213, 215, 217, 220, 222, 223, 224, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241, 242, 243, 244, 247, 248, 250, 252, 253, 255, 256, 257, 258, 259, 260, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 291, 292, 293, 295, 301, 302, 303, 305, 307, 308, 309, 3

In [262]:
eeg_dataset['dataset'][0]['eeg'].size(1)

500

In [228]:
train_dataset = CustomDataset(train_images)
val_dataset = CustomDataset(val_images)
test_dataset = CustomDataset(test_images)

In [280]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

9565
1194
1206


In [229]:
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=16)
# train_loader = DataLoader(train_dataset,shuffle=True,batch_size=16)
# val_loader = DataLoader(val_dataset,num_workers=1,persistent_workers=True)
val_loader = DataLoader(val_dataset,batch_size=16)
test_loader = DataLoader(test_dataset,batch_size=16)

example = next(iter(val_loader))
print(example[0])
print(example[0].shape)
print(example[1])
print(type(example[1]))

tensor([[[-0.4563, -0.4340, -0.3501,  ..., -0.4229, -0.4451, -0.4357],
         [-0.3364, -0.2943, -0.2053,  ..., -0.3653, -0.3573, -0.3356],
         [ 0.2969,  0.3242,  0.3049,  ...,  0.7798,  0.8596,  0.8888],
         ...,
         [ 0.1318,  0.1914,  0.2296,  ...,  0.5548,  0.5299,  0.4644],
         [-0.0015,  0.0017,  0.0045,  ...,  0.0174,  0.0151,  0.0118],
         [-0.0804, -0.0470, -0.0090,  ...,  0.2288,  0.2044,  0.1678]],

        [[-0.2068, -0.1347, -0.0714,  ..., -0.3064, -0.3435, -0.3556],
         [-0.5729, -0.4801, -0.3754,  ..., -0.2887, -0.3335, -0.3484],
         [-0.0833, -0.0024,  0.0727,  ...,  0.6302,  0.5440,  0.4191],
         ...,
         [-0.8683, -0.8660, -0.7844,  ..., -0.1577, -0.1950, -0.1863],
         [-0.0382, -0.0370, -0.0318,  ..., -0.0322, -0.0330, -0.0306],
         [-0.6318, -0.6247, -0.5540,  ..., -0.3484, -0.3780, -0.3549]],

        [[-0.5945, -0.5942, -0.5506,  ..., -0.3239, -0.2993, -0.2503],
         [-0.2989, -0.2890, -0.2421,  ..., -0

In [230]:
len(val_loader.dataset)

1194

# Model 만들기

In [231]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# seed = 130
# seed = 2222222
seed = 136321234

In [234]:
# with classifier attached
class FeatureExtractorNN(L.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        seed_everything(seed,workers=True)
        
        self.input_size = 128
        self.hidden_size = 128
        self.lstm_layers=1
        self.out_size=128
        
        # self.lstm = nn.LSTM(input_size=128,hidden_size=128,num_layers=128)
        self.lstm = nn.LSTM(self.input_size,self.hidden_size,num_layers=self.lstm_layers,batch_first=True)
        self.output = nn.Sequential(
            nn.Linear(in_features=self.hidden_size,out_features=self.out_size),
            nn.ReLU()
        )
        self.classifer=nn.Sequential(
            nn.Linear(in_features=self.out_size,out_features=40),
            # don't use softmax with cross entropy loss
            # nn.Softmax(dim=1)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
        # self.loss_fn = nn.NLLLoss()
        self.training_step_outputs = {"correct_num":0,"loss_sum":0}
        self.validation_step_outputs = {"correct_num":0,"loss_sum":0}
    
    def forward(self,input):
        batch_size = input.size(0)
        lstm_init=(torch.zeros(self.lstm_layers,batch_size,self.hidden_size),torch.zeros(self.lstm_layers,batch_size,self.hidden_size))
        lstm_init = (lstm_init[0].to(device),lstm_init[0].to(device))
        
        input = input.transpose(1,2)
        
        lstm_out, _ = self.lstm(input,lstm_init)
        # tmp_out = lstm_out[:,-1,:] if input.dim()==3 else lstm_out[-1,:]
        tmp_out = lstm_out[:,-1,:]
        out = self.output(tmp_out)
        # print("out shape",out.shape)
        res = self.classifer(out)
        
        return res
    
    def configure_optimizers(self):
        # return optim.SGD(self.parameters(),lr=1e-4,weight_decay=0.1)
        return optim.Adam(self.parameters(),lr=1e-3,weight_decay=0.1)
    
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        x=x.to(device)
        y=y.to(device)
        out = self(x)
        loss = self.loss_fn(out,y)
        
        self.log_dict({"train_loss":loss},prog_bar=True,on_epoch=True)
        preds = out.argmax(dim=1)
        self.training_step_outputs['correct_num']+=(preds==y).sum()
        self.training_step_outputs['loss_sum']+=loss
        return loss

    def on_train_epoch_end(self) -> None:
        num_correct = self.training_step_outputs['correct_num']
        acc = num_correct/len(train_dataset)
        loss = self.training_step_outputs['loss_sum']/len(train_loader)
        print("\n")
        # print("EPOCH:",self.current_epoch)
        print(f"Training accuracy: {acc.item()} ({num_correct.item()}/{len(train_dataset)} correct)")
        print("Training loss (average):",loss.item())
        print("\n")
        self.training_step_outputs['correct_num']=0
        self.training_step_outputs['loss_sum']=0
    
    def validation_step(self,batch,batch_idx):
        x,y = batch
        x=x.to(device)
        y=y.to(device)
        out = self(x)
        loss = self.loss_fn(out,y)

        self.log_dict({"val_loss":loss},prog_bar=True,on_epoch=True)
        preds = out.argmax(dim=1)
        self.validation_step_outputs['correct_num']+=(preds==y).sum()
        self.validation_step_outputs['loss_sum']+=loss
        # return loss
    
    def on_validation_epoch_end(self) -> None:
        num_correct = self.validation_step_outputs['correct_num']
        acc = num_correct/len(val_dataset)
        loss = self.validation_step_outputs['loss_sum']/len(val_loader)
        print('\n')
        # print("EPOCH:",self.current_epoch)
        print(f"Validation accuracy: {acc.item()} ({num_correct.item()}/{len(val_dataset)} correct)")
        print("Validation loss (average):",loss.item())
        print('\n')
        self.validation_step_outputs['correct_num']=0
        self.validation_step_outputs['loss_sum']=0
    
    def test_step(self,batch,batch_idx):
        x,y = batch
        
        out = self(x)
        loss = self.loss_fn(out,y)
        
        y_hat = torch.argmax(out,dim=1)
        # print("OUT,YHAT:",out,y_hat)
        test_acc = torch.sum(y == y_hat).item() / (len(y) * 1.0)
        
        self.log_dict({'test_loss': loss, 'test_acc': test_acc},prog_bar=True,on_epoch=True)
        # print("   ||   test loss:",loss.item(), "   ||   test accuracy:",test_acc )


        

In [235]:
# seed = 136321234
model = FeatureExtractorNN()
model.to(device)
trainer=L.Trainer()
trainer.validate(model,dataloaders=val_loader)


Seed set to 136321234
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ms/anaconda3/envs/neuroimagen/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Validation DataLoader 0: 100%|██████████| 75/75 [00:00<00:00, 85.32it/s]

Validation accuracy: 0.055276382714509964 (66/1194 correct)
Validation loss (average): 3.6390278339385986


Validation DataLoader 0: 100%|██████████| 75/75 [00:00<00:00, 83.41it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    3.6390812397003174     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 3.6390812397003174}]

In [237]:
version_num= 96
epoch = 9
step = 23920
PATH = os.path.join(root_path,"lightning_logs","version_"+str(version_num),"checkpoints","epoch="+str(epoch)+"-step="+str(step)+".ckpt")

model = FeatureExtractorNN()
model.to(device)
# model = FeatureExtractorNN.load_from_checkpoint(PATH)

# outputs = model(train_dataset[0][0])
# print(outputs)

trainer = L.Trainer(max_epochs=100)
# trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader,ckpt_path=PATH)
trainer.validate(model,dataloaders=val_loader)
trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader)
# trainer.fit(model,train_dataloaders=train_loader)
# trainer.validate(model, dataloaders=val_loader)

Seed set to 136321234
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Validation DataLoader 0:  83%|████████▎ | 62/75 [00:00<00:00, 109.12it/s]


  | Name      | Type             | Params
-----------------------------------------------
0 | lstm      | LSTM             | 132 K 
1 | output    | Sequential       | 16.5 K
2 | classifer | Sequential       | 5.2 K 
3 | loss_fn   | CrossEntropyLoss | 0     
-----------------------------------------------
153 K     Trainable params
0         Non-trainable params
153 K     Total params
0.615     Total estimated model params size (MB)


Validation DataLoader 0:  83%|████████▎ | 62/75 [00:00<00:00, 106.77it/s]


RuntimeError: Inplace update to inference tensor outside InferenceMode is not allowed.You can make a clone to get a normal tensor before doing inplace update.See https://github.com/pytorch/rfcs/pull/17 for more details.

# Testing Accuracy

In [170]:
trainer.test(model,dataloaders=test_loader)

Testing DataLoader 0:   1%|▏         | 1/76 [00:00<00:01, 61.13it/s]

Testing DataLoader 0: 100%|██████████| 76/76 [00:00<00:00, 89.34it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            0.0            │
│         test_loss         │     3.689608573913574     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 3.689608573913574, 'test_acc': 0.0}]

In [ ]:
idx = 6
query = val_dataset[idx][0].unsqueeze(dim=0)
# print(query.shape)
query.to(device)
pred = model(query)
# print(pred)
pred = torch.argmax(pred,dim=1)
# pred = pred.max(dim=1)
# print(pred)
print("predicted: ",id_to_name[lookup_dict[pred.item()]])
print("answer: ",id_to_name[lookup_dict[val_dataset[idx][1]]])


RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at mps:0

In [55]:
# Calculate test accuracy
num_correct = 0
model.to(device)
for x,y in (test_loader):
    x=x.to(device)
    y=y.to(device)
    out = model(x)
    y_hat = out.argmax(dim=1)
    # print(y==y_hat)
    num_correct += (y==y_hat).sum()
acc = num_correct/len(test_loader.dataset)
print("Accuracy:",acc.item())

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 64])

In [ ]:
acc

tensor([0.], device='mps:0')